# Modelos pré-treinados pra classificação de normas

In [1]:
%reload_ext autoreload
%autoreload 1

%aimport dataset

import numpy as np
import pandas as pd
import requests
from readability import Document
from sklearn.externals import joblib

### Download dos arquivos do modelo

In [2]:
url_model = 'https://www.dropbox.com/s/sgbcxtlxc85il3q/{}?dl=1'
url_labels = 'https://www.dropbox.com/s/enq14ig757pw2e6/{}?dl=1'
data_root = 'models'

clf_zip = dataset.maybe_download(url_model, 'bag-of-words-all-classes.zip', 396919561, data_root=data_root)
temas_filename = dataset.maybe_download(url_labels, 'temas.csv', 1227, data_root=data_root)
clf_filename = dataset.extract_zip(clf_zip)

Attempting to download: bag-of-words-all-classes.zip



Download Complete!
Found and verified models\bag-of-words-all-classes.zip
Attempting to download: temas.csv



Download Complete!
Found and verified models\temas.csv
File extracted to: models\bag-of-words-all-classes.pkl


### Carregando modelo treinado

In [3]:
clf = joblib.load(clf_filename)
temas = pd.read_csv('models/temas.csv')
print('Modelo treinado para {} temas'.format(temas.shape[0]))

Modelo treinado para 39 temas


In [11]:
#os.environ['http_proxy'] = "http://127.0.0.1:3128" 
#os.environ['https_proxy'] = "https://127.0.0.1:3128"

def get_text_from_url(url):
    '''
    Get most significant text returned from URL.
    '''
    response = requests.get(url)
    doc = Document(response.text)
    text = dataset.preprocess_html(doc.summary())
    return text


def predict_labels(clf, labels, text, top_k=3):
    '''
    Returns top_k labels for text.
    '''
    vectorizer = clf.named_steps['tfidf']
    tfidf = vectorizer.transform([text])
    probs = clf.named_steps['clf'].predict_proba(tfidf)
    
    preds = np.argsort(probs, axis=1)[0,-top_k:][::-1]
    probs = probs[:, preds][0]
    pred_labels = list(map(lambda x: labels[labels.CLASS == x].TEMA.values[0], preds))
    preds_df = pd.DataFrame({'LABEL': pred_labels, 'PROBABILITY': probs})
    
    #if verbose:
    print(preds_df, '\n')
    return preds_df


def predict_labels_url(clf, labels, url, top_k=3, verbose=False):
    text = get_text_from_url(url)
    if verbose:
        print('Text:\n\n{}\n'.format(text))
    return predict_labels(clf, labels, text, top_k), text


def top_tfidf_feats(row, features, top_k=10):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_k]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['TOP TERMS', 'TF_IDF SCORE']
    return df


def top_terms(clf, text, top_k=10):
    ''' Top tfidf features in specific document (matrix row) '''
    
    vectorizer = clf.named_steps['tfidf']
    tfidf = vectorizer.transform([text])    
    row = np.squeeze(tfidf.toarray())
    return top_tfidf_feats(row, vectorizer.get_feature_names(), top_k)


def top_terms_url(clf, url, top_k=10, verbose=False):
    text = get_text_from_url(url)
    if verbose:
        print('Text:\n\n{}\n'.format(text))
    return top_terms(clf, text, top_k), text

### Classificação de normas

In [13]:
url = 'http://www2.camara.leg.br/legin/fed/lei/1990/lei-8112-11-dezembro-1990-322161-publicacaooriginal-1-pl.html'
%time preds, text = predict_labels_url(clf, temas, url)
%time terms = top_terms(clf, text)
print('\n\n')
print(terms)

                              LABEL  PROBABILITY
0             Administração Pública     0.646957
1  Previdência e Assistência Social     0.158658
2  Direito Penal e Processual Penal     0.032886 

Wall time: 903 ms
Wall time: 3.77 s



     TOP TERMS  TF_IDF SCORE
0     servidor      0.376633
1           ou      0.271193
2        cargo      0.175954
3           ss      0.155810
4      licenca      0.142387
5         sera      0.139701
6  do servidor      0.121269
7          por      0.099847
8           se      0.095868
9           ao      0.094658


In [14]:
url = 'http://www2.camara.leg.br/legin/fed/decret/2017/decreto-9060-26-maio-2017-784946-publicacaooriginal-152840-pe.html'
%time preds, text = predict_labels_url(clf, temas, url)
%time terms = top_terms(clf, text)
print('\n\n')
print(terms)

                                  LABEL  PROBABILITY
0  Organização Administrativa do Estado     0.437761
1                 Administração Pública     0.367108
2                  Ciência e Tecnologia     0.104777 

Wall time: 655 ms
Wall time: 3.77 s



                      TOP TERMS  TF_IDF SCORE
0                     inovacoes      0.246960
1                    da ciencia      0.153009
2                        gestao      0.152558
3                    tecnologia      0.152340
4        desenvolvimento gestao      0.152133
5                  comunicacoes      0.151626
6            ciencia tecnologia      0.151108
7                    ministerio      0.132678
8  planejamento desenvolvimento      0.132676
9                  diretoria de      0.126855


### Classificação de notícias

In [16]:
url = 'http://www2.camara.leg.br/camaranoticias/noticias/538631.html'
%time preds, text = predict_labels_url(clf, temas, url, verbose=True)
%time terms = top_terms(clf, text)
print('\n\n')
print(terms)

Text:

A comissao especial que analisa a proposta (PEC 15/15) que torna permanente o Fundo de Manutencao e Desenvolvimento da Educacao Basica e Valorizacao dos Profissionais da Educacao (Fundeb) promove audiencia publica nesta manha para discutir sugestoes de aprimoramento do texto e medidas de cooperacao entre os entes federados no setor educacional.

Criado em 2006 para vigorar ate 2020, o Fundeb e um fundo que utiliza recursos federais, dos estados, Distrito Federal e municipios para financiar a educacao basica no Pais, incluindo a remuneracao dos professores.
O evento e uma iniciativa da relatora da PEC, deputada Professora Dorinha Seabra Rezende (DEM-TO). Segundo ela, com o fim da vigencia do fundo se aproximando e urgente debater o assunto.
Foram convidados para a reuniao: o vice-presidente da Frente Nacional de Prefeitos (FNP), Elias Dinis; e o gestor do Observatorio de Informacoes Municipais, Francois Eugene Jean de Bremaeker.
A audiencia sera realizada no plenario 9, a partir 

In [17]:
url = 'http://g1.globo.com/sao-paulo/noticia/temer-diz-que-governo-estuda-aumento-da-aliquota-do-ir-mas-que-ainda-nao-ha-nada-decidido.ghtml'
%time preds, text = predict_labels_url(clf, temas, url, verbose=True)
%time terms = top_terms(clf, text)
print('\n\n')
print(terms)

Text:

  Temer diz que governo estuda aumento da aliquota do IR, mas que ainda nao ha 'nada decidido'   'Ha estudos, dos mais variados estudos', disse em evento em Sao Paulo. Medidas precisam ser aprovadas pelo Congresso.              Temer diz que governo estuda aumento da aliquota do IR, mas que ainda nao ha 'nada decidi        O presidente Michel Temer (PMDB) afirmou nesta terca-feira (8) que ha estudos sendo feitos para o aumento da aliquota do Imposto de Renda (IR), mas que nao ha nada definido. A equipe economica quer aumentar a arrecadacao - ate junho, o deficit das contas do governo era de R$ 56,092 bilhoes.     "Ha estudos, ha dos mais variados estudos. Sao estudos que se fazem rotineiramente. A todo momento estao fazendo planejamento nos setores da economia, eles fazem esses estudos. Sao estudos que estao sendo feitos, mas nada decidido", disse o presidente apos cerimonia de abertura da Fenabrave em Sao Paulo.     Parte da equipe economica decidiu tirar da gaveta projetos de 

In [18]:
url = 'http://www2.camara.leg.br/camaranoticias/noticias/DIREITOS-HUMANOS/538744-COMISSAO-REDUZ-IDADE-PARA-PESSOA-COM-DEFICIENCIA-SER-DECLARADA-IDOSA.html'
%time preds, text = predict_labels_url(clf, temas, url, verbose=True)
%time terms = top_terms(clf, text)
print('\n\n')
print(terms)

Text:



Lucio Bernardo Junior / Camara dos Deputados

Mara Gabrilli:  as limitacoes tipicas do envelhecimento aparecem antes nas pessoas com deficiencia

A Comissao de Defesa dos Direitos das Pessoas com Deficiencia aprovou proposta (PL 1118/11) que altera a idade para que a pessoa com deficiencia seja considerada idosa. Pelo texto aprovado, a pessoa com deficiencia passara a ser considerada idosa aos 50 anos.  

A proposta permite ainda a reducao desse limite, mediante avaliacao da deficiencia, feita a partir de laudos biopsicossociais. Os laudos deverao ser elaborados por medicos juntamente com especialistas que avaliam as barreiras sociais e psicologicas enfrentadas pela pessoa.
O texto original, do deputado Eduardo Barbosa (PSDB-MG), estabelecia a idade em 45 anos, mas o projeto foi alterado pela relatora na Comissao de Seguridade Social, deputada Carmen Zanotto (PPS-SC), para os 50 anos. Na Comissao de Defesa dos Direitos das Pessoas com Deficiencia a relatora, deputada Mara Gabr

In [19]:
url = 'http://www2.camara.leg.br/camaranoticias/noticias/TRABALHO-E-PREVIDENCIA/538706-COMISSAO-DE-TRABALHO-APROVA-A-REGULAMENTACAO-DA-PROFISSAO-DE-GARCOM-COM-PISO-SALARIAL-DE-R$-2.811.html'
%time preds, text = predict_labels_url(clf, temas, url, verbose=True)
%time terms = top_terms(clf, text)
print('\n\n')
print(terms)

Text:



Edvaldo Belitardo / Camara dos Deputados

Felipe Bornier propos o piso salarial para os garcons profissionais

A Comissao de Trabalho, de Administracao e Servico Publico aprovou, com duas emendas, proposta que regulamenta a profissao de garcom - reconhecido no texto como aquele que, em estabelecimentos do ramo de hoteis, restaurantes, bares e similares, exerce a funcao de servir alimentos e bebidas a clientes.
O texto aprovado assegura ao garcom o direito a um piso salarial no valor de R$ 2.811,00, para uma jornada de 8 horas diarias, devendo as horas extraordinarias serem pagas com 50% de acrescimo sobre o salario legal ou contratual.
O texto original - Projeto de Lei 6227/13, do deputado Wilson Filho (PMDB-PB) - previa um piso salarial de tres salarios minimos e horas extras pagas com acrescimo de 30%.
Duas emendas
Ao analisar a proposta, no entanto, o relator na comissao, deputado Felipe Bornier (Pros-RJ), apresentou duas emendas. Bornier lembrou que a Constituicao Federal 